In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.models import ModelCatalog


import ray
from ray import tune
from ray.tune import track

import math
import gym

from gym_compete_to_rllib import created_envs, env_name, create_env, env_name_rllib

import os
os.environ['DISPLAY'] = ':0'

In [2]:
def ray_init():
    ray.shutdown()
    return ray.init(ignore_reinit_error=True,
                    temp_dir='/scratch/sergei/tmp') # Skip or set to ignore if already called

ray_init()


2020-07-17 22:29:35,690	INFO resource_spec.py:223 -- Starting Ray with 149.41 GiB memory available for workers and up to 68.05 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-17 22:29:36,539	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.32.175.10',
 'raylet_ip_address': '128.32.175.10',
 'redis_address': '128.32.175.10:39093',
 'object_store_address': '/scratch/sergei/tmp/session_2020-07-17_22-29-35_659013_3201/sockets/plasma_store',
 'raylet_socket_name': '/scratch/sergei/tmp/session_2020-07-17_22-29-35_659013_3201/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/scratch/sergei/tmp/session_2020-07-17_22-29-35_659013_3201'}

In [3]:
tf.keras.backend.set_floatx('float32')

In [6]:
env_cls = create_env
env_config = {'with_video': True}

def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "player_%d"

    def get_agent_config(agent_id):
        agent_config = (PPOTFPolicy, obs_space, act_space, {
            'model': {
                        "custom_model": "GymCompetePretrainedModel",
                        "custom_model_config": {
                            "agent_id": agent_id - 1,
                            "env_name": env_name,
                            "model_config": {},
                            "name": "model_%s" % (agent_id - 1)
                        },           
                        
                    },
            
            "framework": "tfe",
        })
        
        return agent_config
        
#         agent_config = (PPOTFPolicy, obs_space, act_space, {
#                     "model": {
#                         "use_lstm": False,
#                         "fcnet_hiddens": [64, 64],
#                         #"custom_action_dist": "DiagGaussian",
#                     },
#                     "framework": "tfe",
#                 })
        
#         return agent_config

    N_POLICIES = 2

    policies = {policy_template % i: get_agent_config(i) for i in range(1, 1  + N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player_1", "player_2"]
        agent_ids = ["player_1", "player_2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_name_rllib,
        "env_config": env_config,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
       "train_batch_size": config['train_batch_size'],
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    print("Using config")
    print(config)
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

def train(trainer, stop_iters, do_track=True):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        if do_track:
            track.log(**results)
    o = trainer.save_to_object()
    return o

trainer = None

def train_one(config, restore_state=None, do_track=True):
    print(config)
    rl_config = build_trainer_config(restore_state=restore_state,
                              train_policies=config['train_policies'],
                              config=config)
    global trainer
    trainer = build_trainer(restore_state=None, config=rl_config)
    train(trainer, config['train_steps'], do_track=do_track)


# try changing learning rate
config = {'train_batch_size': 4096}

config['train_steps'] = 100

# ['humanoid_blocker', 'humanoid'],
config['train_policies'] = []
config['num_workers'] = 32

In [7]:
train_one(config, do_track=False)

{'train_batch_size': 4096, 'train_steps': 100, 'train_policies': [], 'num_workers': 32}
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-c

2020-07-17 22:30:11,246	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
observation_preprocessing_la multiple                  761       
_________________________________________________________________
h1 (Dense)                   multiple                  24384     
_________________________________________________________________
h2 (Dense)                   multiple                  4160      
_________________________________________________________________
value (Dense)                multiple                  65        
_________________________________________________________________
value_postprocessing_layer_3 multiple                  2         
Total params: 29,372
Trainable params: 28,609
Non-trainable params: 763
_________________________________________________________________
Weights delta 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, th

/scratch/sergei/miniconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has shape (), but the VideoRecorder is configured for shape (1080, 1920, 3).
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


KeyboardInterrupt: 

In [ ]:
[x.close() for x in created_envs]

In [9]:
dir(trainer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allow_unknown_configs',
 '_allow_unknown_subkeys',
 '_before_evaluate',
 '_default_config',
 '_env_id',
 '_episodes_total',
 '_evaluate',
 '_experiment_id',
 '_export_model',
 '_init',
 '_is_overriden',
 '_iteration',
 '_iterations_since_restore',
 '_local_ip',
 '_log_result',
 '_logdir',
 '_make_workers',
 '_monitor',
 '_name',
 '_override_all_subkeys_if_type_changes',
 '_policy',
 '_register_if_needed',
 '_restore',
 '_restored',
 '_result_logger',
 '_save',
 '_setup',
 '_stop',
 '_sync_filters_if_needed',
 '_time_since_restore',
 '_time_total',
 '_timesteps_since_restore',
 '_timeste

In [15]:
created_envs[-1].video_recorder.path

'/scratch/sergei/better-adversarial-defenses/video-20200717-223009-be561228-c8b7-11ea-8b6f-1831bf455a80/video.000000.mp4'

In [ ]:
from load_gym_compete_policy import get_policy_value_nets

In [ ]:
obs = env_cls(env_config).reset()

In [ ]:
acts = [trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)]

In [ ]:
np.mean(acts, axis=0)

In [ ]:
nets = get_policy_value_nets(env_name, 0)
policy_net_orig = nets['policy_mean_logstd_flat']

In [ ]:
m = trainer.get_policy('player_1').model
m.policy_net(obs['player_1'].reshape(1, -1))

In [ ]:
nets['policy_mean_logstd'](obs['player_1'].reshape(1, -1))

In [ ]:
mean, logstd = tf.split(policy_net_orig(obs['player_1'].reshape(1, -1)), 2, axis=1)

In [ ]:
np.mean([np.clip(np.random.normal(loc=mean[0], scale=np.exp(logstd[0]), size=(17,)), -0.4, 0.4) for _ in range(1000)], axis=0)

In [ ]:
np.mean([trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)], axis=0)

In [ ]:
trainer.compute_action(obs['player_1'], policy_id='player_1')

In [ ]:
env_cls(env_config).action_space.high